In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tmalik8\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with open('intents.json') as file:
    data = json.load(file)

In [4]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do?',
    'What help you 

In [5]:
words = []
labels = []
docs_x = []
docs_y = []
ignore_words = ['?']

# loop through each sentence in our intents patterns
for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        wrds = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(wrds)
        # add to documents in our corpus
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
        # add to our classes list
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [6]:
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [7]:
# remove duplicates
labels = sorted(list(set(labels)))

In [8]:
print (len(docs_x), "documents")
print (len(labels), "classes", labels)
print (len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
82 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [9]:
# create our training data
training = []
output = []
# create an empty array for our output
out_empty = [0 for _ in range(len(labels))]

In [10]:
# training set, bag of words for each sentence
for x,doc in enumerate(docs_x):
    # initialize our bag of words
    bag = []
    # stem each word
    wrds = [stemmer.stem(w) for w in doc]
    # create our bag of words array
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [11]:
training = np.array(training)
output = np.array(output)

In [12]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

In [13]:
# Define model and setup tensorboard
model = tflearn.DNN(net)

In [14]:
# Start training (apply gradient descent algorithm)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 5999  | total loss: 0.00890 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.00890 - acc: 1.0000 -- iter: 40/45
Training Step: 6000  | total loss: 0.00874 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.00874 - acc: 1.0000 -- iter: 45/45
--


In [15]:
model.save("model.tflearn")

INFO:tensorflow:C:\Users\tmalik8\ChatBot from Scratch\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

In [19]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.8:

            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print("Chatbot:",random.choice(responses))
        else:
            print("Chatbot: I didn't understand. Try again with a different question")

In [21]:
chat()

Start talking with the bot (type quit to stop)!
You: hello
Chatbot: Good to see you again
You: hi
Chatbot: Hello, thanks for asking
You: how can u help me
Chatbot: I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies
You: asjkdkl
Chatbot: I didn't understand. Try again with a different question
You: thanks
Chatbot: Happy to help!
You: bye
Chatbot: Have a nice day
You: quit
